# Import

In [20]:
import json
import itertools
from functools import reduce

import pandas as pd

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 8

# Function definition

In [21]:
def save_fig(fig, name):
    fig.savefig("./graphs/" + name + ".pdf", format="pdf")

In [22]:
def fix_status(df_line):
    if df_line.status != "UNKNOWN":
        return df_line.status

    if df_line.objectives != {}:
        return "SATISFIABLE"

    return df_line.status

Borda counting

|     s1 \ s2 | satisfied | unsatisfied |     unkown | optimized |
|-------------|-----------|-------------|------------|-----------|
|   satisfied | o, timing |         bug |        s_1 |       s_2 |
| unsatisfied |       bug |      timing |        s_1 |       bug |
|      unkown |       s_2 |         s_2 |        0-0 |       s_2 |
|   optimized |       s_1 |         bug |        s_1 |    timing |

where
- s_x means give one point to solver x and zero to the other solver
- o mean look at the objective value. If one solver gives a better answer, that
  solver wins one point and the other zero. If both are equals, look at the
  times
- bug means one of the two solver contains a bug
- timing mean look directly at the times
- 0-0 means that each solver gets zero points

In [23]:
def score(t1, t2):
    if t1 == 0 and t2 == 0:
        return (0.5, 0.5)
    else:
        return (
            t2 / (t1 + t2),
            t1 / (t1 + t2),
        )

def borda_count(x):
    try:
        model1 = x.iloc[0]
        model2 = x.iloc[1]

        status1 = model1.status
        status2 = model2.status

        obj1 = max(model1.objectives.values(), default=None)
        obj2 = max(model2.objectives.values(), default=None)

        t1 = int(model1.solve_time - model1.flat_time)
        t2 = int(model2.solve_time - model2.flat_time)
    except:
        print(x)

    if status1 == "Optimal solution":
        if status2 == "Optimal solution":
            if obj1 == obj2:
                return score(t1, t2)
            else:
                print("ERROR: BOTH SOLVERS SAY THEY REACHED OPTIMAL SOLUTION BUT THE VALUES ARE DIFFERENT")
                pass
        elif status2 == "Unsatisfiable":
            print("ERROR: ONE SOLVER REACHED THE OPTIMAL SOLUTION BUT THE OTHER ONE SAY UNSAT")
            pass
        else:
            return (1, 0)
    elif status1 == "Unknown":
        if status2 == "Unknown":
            return (0, 0)
        else:
            return (0, 1)
    elif status1 == "Unsatisfiable":
        if status2 == "Satisfiable" or status2 == "Optimal solution":
            print("ERROR: ONE SOLVER REACHED UNSAT BUT THE OTHER ONE SAY SAT/OPTIMAL")
            pass
        elif status2 == "Unknown":
            return (1, 0)
        else:
            return score(t1, t2)
    elif status1 == "Satisfiable":
        if status2 == "Unknown":
            return (1, 0)
        elif status2 == "Optimal solution":
            return (0, 1)
        elif status2 == "Satisfiable":
            if obj1 > obj2: # Minimization problem
                return (0, 1)
            elif obj1 < obj2:
                return (1, 0)
            else:
                return score(t1, t2)
        else:
            print("ERROR: ONE SOLVER REACHED SAT BUT THE OTHER ONE SAY UNSAT")
            pass

    # Exit results for errors
    errors.append(x)
    return (0, 0)


# Dataframe import

In [24]:
with open("./results.json", "r") as f:
    results = json.load(f)

dfs = []
for k, v in results.items():
    x = pd.DataFrame(v).assign(**{
        "solver" : k
    })
    dfs.append(
        x
    )

df = pd.concat(dfs).assign(**{
    "test" : lambda df : df.problem.apply(lambda x : int(x.split("/")[-1].split("-")[1][:2])),
    "solve_time" : lambda df : df.apply(
        lambda x :
            300
            if "UNKNOWN" in x["status"]
            else x.solve_time,
        axis=1),
    "status" : lambda df : df.apply(
        lambda x : fix_status(x),
    axis=1)
}).replace({
    "cpsatlp": "OR-Tools",
    "gecode": "Gecode",
    "highs": "Highs",
    "chuffed": "Chuffed",
    "OPTIMAL_SOLUTION": "Optimal solution",
    "UNSATISFIABLE": "Unsatisfiable",
    "UNKNOWN": "Unknown",
    "SATISFIABLE": "Satisfiable"
}).drop(
    columns=[]
).assign(optimal_solution=lambda df : df.apply(
    lambda x : max(x.objectives.values()) if "Optimal solution" in x.status else None, axis=1
)).reset_index(drop=True)

df

,problem,status,flat_time,solve_time,objectives,solver,test,optimal_solution
0,TOIT/data/data-82.dzn,Optimal solution,0.206,0.625,"{'0.611': 18, '0.616': 25}",Chuffed,82,25.0
1,TOIT/data/data-96.dzn,Unsatisfiable,0.236,0.571,{},Chuffed,96,NaN
2,TOIT/data/data-69.dzn,Optimal solution,0.318,0.894,{'0.831': 28},Chuffed,69,28.0
3,TOIT/data/data-41.dzn,Unsatisfiable,0.364,0.651,{},Chuffed,41,NaN
4,TOIT/data/data-55.dzn,Optimal solution,0.367,1.036,"{'1.003': 21, '1.011': 35}",Chuffed,55,35.0
...,...,...,...,...,...,...,...,...
395,TOIT/data/data-71.dzn,Unsatisfiable,0.185,0.199,{},Highs,71,NaN
396,TOIT/data/data-59.dzn,Optimal solution,0.192,0.258,"{'0.216': 35, '0.232': 39, '0.242': 40}",Highs,59,40.0
397,TOIT/data/data-58.dzn,Unsatisfiable,0.180,0.192,{},Highs,58,NaN
398,TOIT/data/data-70.dzn,Unsatisfiable,0.187,0.274,{},Highs,70,NaN


# Clean Up

In [25]:
# Remove instances that not all solvers has run
errors = []
for _, g in df.groupby("test"):
    if len(g) != df.solver.nunique():
        errors.append(g.reset_index().assign(reason="less than four solver run this instance"))
        continue
errors = pd.concat(errors) if len(errors) > 0 else pd.DataFrame()
print("Erros amount: ", len(errors))

Erros amount:  0


In [26]:
# Remove instances with faulty results between solvers
faulty_result = []
for _, g in df.groupby("test"):
    if len(g.status.value_counts().reset_index().query("status == 'Unsatisfiable'")) > 0:
        if len(g.status.value_counts().reset_index().query("status != 'Unsatisfiable'")) > 0:
            faulty_result.append(g)
print("Erros amount: ", len(faulty_result))


Erros amount:  0


In [27]:
solvers = df.solver.unique()

# DataFrame Analysis

In [28]:
df.status.unique()

array(['Optimal solution', 'Unsatisfiable'], dtype=object)

In [29]:
df.status.value_counts(normalize=True)

status
Optimal solution    0.54
Unsatisfiable       0.46
Name: proportion, dtype: float64

In [30]:
df[["solver", "status"]].groupby("solver").value_counts().reset_index()

,solver,status,count
0,Chuffed,Optimal solution,54
1,Chuffed,Unsatisfiable,46
2,Gecode,Optimal solution,54
3,Gecode,Unsatisfiable,46
4,Highs,Optimal solution,54
5,Highs,Unsatisfiable,46
6,OR-Tools,Optimal solution,54
7,OR-Tools,Unsatisfiable,46


In [31]:
df.solve_time.max()

np.float64(5.848)

# Paper tables

In [32]:
mean_solve_time = (
    df[["solver", "solve_time"]]
    .groupby("solver")
    .aggregate({
        "solve_time": ("mean", "min", "max"),
    })
    .map(lambda x : str(round(x, 2)) + "s")
)
mean_solve_time.columns = [col[1] for col in mean_solve_time.columns.values]
mean_solve_time = mean_solve_time.reset_index().sort_values("solver").rename(columns={"mean": "avg"})
mean_solve_time

,solver,avg,min,max
0,Chuffed,0.47s,0.27s,5.85s
1,Gecode,0.17s,0.14s,0.26s
2,Highs,0.36s,0.17s,2.26s
3,OR-Tools,0.17s,0.14s,0.23s


In [33]:
mean_flat_time = (
    df[["solver", "flat_time"]]
    .groupby("solver")
    .aggregate({
        "flat_time": "mean",
    })
    .rename(columns={"flat_time": "flat"})
    .map(lambda x : str(round(x, 2)) + "s")
    .reset_index()
    .sort_values("solver")
)
mean_flat_time

,solver,flat
0,Chuffed,0.15s
1,Gecode,0.13s
2,Highs,0.17s
3,OR-Tools,0.13s


In [34]:
first_last_solve_time = (
    df[["solver", "objectives"]]
    .assign(**{
        "objectives_min" : lambda df: df.objectives.apply(
            lambda x : min([float(e) for e in x.keys()], default=pd.NA)
        ),
        "objectives_max" : lambda df: df.objectives.apply(
            lambda x : min([float(e) for e in x.keys()], default=pd.NA)
        )
    })
    .dropna()
    .groupby("solver")
    .agg({"objectives_min": "mean", "objectives_max": "mean"})
    .rename(columns={"objectives_min": "first", "objectives_max": "last"})
    .map(lambda x : str(round(x, 2)) + "s")
    .reset_index()
    .sort_values("solver")
)
first_last_solve_time

,solver,first,last
0,Chuffed,0.36s,0.36s
1,Gecode,0.15s,0.15s
2,Highs,0.28s,0.28s
3,OR-Tools,0.17s,0.17s


In [35]:
scores = []
errors = []
for solver1, solver2 in itertools.combinations(solvers, r=2):
    to_group = df.query("solver == @solver1 or solver == @solver2")
    for _, group in to_group.groupby("test"):
        if len(group) == 2: # Make sure that if an instance failed to run, this code will run anyway
            scores.append(group.assign(score=borda_count))

scores = (
    pd.concat(scores)
    [["solver", "score"]]
    .groupby("solver")
    .sum()
    .map(lambda x : str(round(x, 2)))
    .reset_index()
    .rename(columns={"score": "Score"})
)

In [36]:
scores

,solver,Score
0,Chuffed,150.0
1,Gecode,152.0
2,Highs,146.0
3,OR-Tools,152.0


In [37]:
print(reduce(lambda left, right: pd.merge(left, right, on=['solver']), [
    mean_solve_time,
    mean_flat_time,
    first_last_solve_time,
    scores
]).sort_values("Score", ascending=False).to_latex(multirow=False, index=False))

\begin{tabular}{llllllll}
\toprule
solver & avg & min & max & flat & first & last & Score \\
\midrule
Gecode & 0.17s & 0.14s & 0.26s & 0.13s & 0.15s & 0.15s & 152.0 \\
OR-Tools & 0.17s & 0.14s & 0.23s & 0.13s & 0.17s & 0.17s & 152.0 \\
Chuffed & 0.47s & 0.27s & 5.85s & 0.15s & 0.36s & 0.36s & 150.0 \\
Highs & 0.36s & 0.17s & 2.26s & 0.17s & 0.28s & 0.28s & 146.0 \\
\bottomrule
\end{tabular}

